In [ ]:
%cd drive/MyDrive/data/aT

/content/drive/MyDrive/data/aT


# Import

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import Dataset, DataLoader

import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# torch.random.manual_seed(seed=2020320120)

# Dataset
데이터셋을 만들 때 고려해야 할 조건
* train, val, test를 어떻게 구분할 것인가? -> 먼저, split 인자를 통해 (train, val), test를 구분한다.
* 특히 train, val을 어떻게 구분할 것인가? -> phase 인자를 통해 train, val을 구분한다.
* 품목에 따라 데이터셋을 만들 것인가? -> use_all_items 인자와 item_idx 인자를 통해 모든 품목을 사용할 것인지, 하나의 품목만 사용할 것인지, 하나의 품목만 사용할 것이라면 사용할 품목의 인덱스는 무엇인지 설정한다.

In [ ]:
def time_window(df, t, t_sep, use_scale=False):
    seq_len = t
    seqence_length = seq_len + t_sep

    scaler = MinMaxScaler()

    result = []
    for index in range(len(df) - seqence_length):
        data = np.array(df[index: index + seqence_length].values, dtype=np.float32)
        if use_scale:
            scaler.fit(data)
            data = scaler.transform(data)
        result.append(data)
    return np.array(result)

In [ ]:
class NongDataset(Dataset):
    def __init__(self, split, use_all_pummok=True, idx_pummok=0):
        super().__init__()

        # 'train' or 'test'
        self.split = split

        # initialize data
        if self.split == 'train':
            self.init_train(use_all_pummok, idx_pummok)
        elif self.split == 'test':
            self.init_test(use_all_pummok, idx_pummok)
        else:
            raise ValueError()
    
    def init_train(self, use_all_pummok, idx_pummok):
        # in case of train, self.phase is needed
        self.phase = 'train'

        # train_0.csv, train_1.csv, train_2.csv etc.
        self.file_list = [os.path.join(self.split, x) for x in os.listdir(self.split)]

        # columns doesn't used in train phase
        tr_del_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 ']
        
        # Data
        self.data = {
            'train': {
                'inputs': [],
                'labels': []
            },
            'val': {
                'inputs': [],
                'labels': []
            },
        }

        # transform all data
        for file_path in self.file_list:
            data = pd.read_csv(file_path)

            for j in data.columns:
                data[j] = data[j].replace({' ': np.nan})
            
            data.drop(tr_del_list, axis=1, inplace=True)
            data.set_index('datadate', drop=True, inplace=True)

            data = data.fillna(0)

            x, y = data[[i for i in data.columns if i != '해당일자_전체평균가격(원)']], data['해당일자_전체평균가격(원)']

            y = y[14:]

            data_x = time_window(x, 13, 1, use_scale=True)
            data_y = time_window(y, 27, 1)

            xdata = data_x[:len(data_y)]
            ydata = data_y

            x_train, x_val, y_train, y_val = train_test_split(xdata, ydata, test_size=0.2, shuffle=False, random_state=119)

            self.data['train']['inputs'].append(x_train)
            self.data['train']['labels'].append(y_train)
            self.data['val']['inputs'].append(x_val)
            self.data['val']['labels'].append(y_val)
        
        self.data['train']['inputs'] = torch.tensor(np.concatenate(self.data['train']['inputs'], axis=0, dtype=np.float32))
        self.data['train']['labels'] = torch.tensor(np.concatenate(self.data['train']['labels'], axis=0, dtype=np.float32))
        self.data['val']['inputs'] = torch.tensor(np.concatenate(self.data['val']['inputs'], axis=0, dtype=np.float32))
        self.data['val']['labels'] = torch.tensor(np.concatenate(self.data['val']['labels'], axis=0, dtype=np.float32))
        
    def init_test(self, use_all_pummok, idx_pummok): # 수정해주세용
        self.path_list = [os.path.join(self.split, x) for x in os.listdir(self.split)]
        self.file_list = []

        # test 에서 사용하지 않는 열
        ts_del_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 ', '해당일자_전체평균가격(원)']

        for path in self.path_list:
            file_list = [os.path.join(path, x) for x in os.listdir(path)]
            self.file_list.append(file_list)


        self.data = {
            'inputs': [],
            'labels': [],
        }

        for file_paths in self.file_list:
            for file_path in file_paths:
                data = pd.read_csv(file_path)

                for j in data.columns:
                    data[j] = data[j].replace({' ': np.nan})

                data.drop(ts_del_list, axis=0, inplace=True)
                data.set_index('datadate', drop=True, inplace=True)

                data = data.fillna(0)

                x, y = data[[i for i in data.columns if i != '해당일자_전체평균가격(원)']], data['해당일자_전체평균가격(원)']

                y = y[14:]

                data_x = time_window(x, 13, 1)
                data_y = time_window(y, 27, 1)

                xdata = data_x[:len(data_y)]
                ydata = data_y

                self.data['inputs'].append(xdata)
                self.data['labels'].append(ydata)

        self.data['inputs'] = np.concatenate(self.data['inputs'], axis=0)
        self.data['labels'] = np.concatenate(self.data['labels'], axis=0)

        print(self.data['inputs'].shape)
        print(self.data['labels'].shape)

    def __getitem__(self, idx):
        if self.split == 'train':
            return self.train_item(idx)
        elif self.split == 'test':
            return self.test_item(idx)
        else:
            raise ValueError()

    def train(self):
        self.phase = 'train'
    
    def eval(self):
        self.phase = 'val'
    
    def train_item(self, idx):
        return self.data[self.phase]['inputs'][idx], self.data[self.phase]['labels'][idx]

    def test_item(self, idx): # 수정하기
        pass
    
    def __len__(self):
        return self.data[self.phase]['inputs'].shape[0]

Model

In [ ]:
class Nongransformer(nn.Module):
    def __init__(self, d_model, n_heads, dropout, n_transformer_blocks):
        super().__init__()

        self.dim_magnification = nn.Sequential(
            nn.Linear(49, d_model // 2),
            nn.ReLU(),
            nn.Linear(d_model // 2, d_model),
            nn.ReLU(),
            nn.Linear(d_model, d_model)
        )

        self.encoder = nn.TransformerEncoder(
            encoder_layer=nn.TransformerEncoderLayer(
                d_model=d_model,
                nhead=n_heads,
                dropout=dropout,
            ),
            num_layers=n_transformer_blocks,
        )

        self.predictor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 14, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 28),
            nn.ReLU()
        )

    def forward(self, inputs):
        dim_magnification = self.dim_magnification(inputs)
        encodings = self.encoder(dim_magnification)
        return self.predictor(encodings)

Train


In [ ]:
def grad_check_fn(named_params):
    params = named_params[1]
    with torch.no_grad():
        sum = params * params
    return sum.sum()

In [ ]:
t = torch.tensor([[1, 2, 3],
                  [1, 2, 3],
                  [1, 2 ,3]] ,dtype=torch.float32)

print(grad_check_fn(('name', t)))

tensor(42.)


In [ ]:
net = Nongransformer(
        d_model=512,
        n_heads=8,
        dropout=0.5,
        n_transformer_blocks=6
    )

n = grad_check_fn(next(net.named_parameters()))

print(f'n: {n}')

if n > 0:
    print('n > 0')

if n < 0:
    print('n < 0')

n: 85.93367004394531
n > 0


In [ ]:
def train_one_epoch(net, dataloader, optimizer, criterion, scheduler, device):
    dataloader.dataset.train()
    net.train()

    total_size = len(dataloader.dataset)

    total_loss = 0.0

    is_grad_update = None

    for inputs, labels in tqdm(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.set_grad_enabled(True):
            outputs = net(inputs)

            loss = criterion(outputs, labels)

        optimizer.zero_grad()

        loss.backward()

        grad_check = grad_check_fn(next(net.named_parameters())).item()
        if is_grad_update == None and grad_check > 0:
            is_grad_update = grad_check

        optimizer.step()

        scheduler.step()

        total_loss += loss.item() * inputs.shape[0]
    
    print(f'Train Total Loss: {total_loss / total_size}')
    
    if is_grad_update:
        print('gradient flow good')
    else:
        print('gradient flow bad')
    
    return net.state_dict()

In [ ]:
def eval_one_epoch(net, dataloader, criterion, device):
    dataloader.dataset.eval()
    net.eval()

    total_size = len(dataloader.dataset)

    total_loss = 0.0

    for inputs, labels in tqdm(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.set_grad_enabled(False):
            outputs = net(inputs)

            loss = criterion(outputs, labels)

        total_loss += loss.item() * inputs.shape[0]
    
    print(f'Eval Total Loss: {total_loss / total_size}')
    
    return total_loss / total_size

In [ ]:
train_dataset = NongDataset('train')

In [ ]:
def train(epochs, train_dataset):
    # Dataset
    # train_dataset = NongDataset('train')
    print('Make Dataset ' + '=' * 30)

    # DataLoader
    train_dataloader = DataLoader(
        dataset=train_dataset,
        batch_size=128,
        shuffle=True
    )
    print('Make DataLoader ' + '=' * 30)

    for inputs, labels in train_dataloader:
        print(f'type(inputs): {type(inputs)}')
        print(f'type(labels): {type(labels)}')
        print(f'inputs.shape: {inputs.shape}')
        print(f'labels.shape: {labels.shape}')

        break

    # Net
    nongransformer = Nongransformer(
        d_model=512,
        n_heads=8,
        dropout=0.5,
        n_transformer_blocks=6
    )
    print('Make Nongransformer ' + '=' * 30)

    for name, param in nongransformer.named_parameters():
        nn.init.normal_(param)
    print('Complete Param Init ' + '=' * 30)
    
    # Criterion
    criterion = nn.L1Loss()

    # Optimizer
    optimizer = optim.Adam(nongransformer.parameters(), lr=1e-3, betas=(0.9, 0.999))

    # Learning Rate Scheduler
    scheduler = lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

    # Device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    nongransformer = nongransformer.to(device)

    best_loss = None
    best_state = None
    update_count = 0
    no_update_count = 0
    late_update = False

    for epoch in range(epochs):
        print(f'{epoch} Epoch ' + '=' * 50)

        if late_update:
            nongransformer.load_state_dict(best_state)
            update_count = update_count + 1
            print(f'Update Count: {update_count} / {update_count + no_update_count}')
        else:
            no_update_count = no_update_count + 1
            print(f'No Update Count: {no_update_count} / {update_count + no_update_count}')

        state = train_one_epoch(
            net=nongransformer,
            dataloader=train_dataloader,
            optimizer=optimizer,
            criterion=criterion,
            scheduler=scheduler,
            device=device
        )

        eval_loss = eval_one_epoch(
            net=nongransformer,
            dataloader=train_dataloader,
            criterion=criterion,
            device=device
        )

        late_update = False

        if best_loss == None:
            best_loss = eval_loss
            best_state = state
            late_update = True
            print(f'Best Loss: {best_loss}')
        elif best_loss > eval_loss:
            best_loss = eval_loss
            best_state = state
            late_update = True
            print(f'Best Loss: {best_loss}')
        else:
            print('No Update')

train(epochs=1000, train_dataset=train_dataset)

Make Dataset ==============================
Make DataLoader ==============================
type(inputs): <class 'torch.Tensor'>
type(labels): <class 'torch.Tensor'>
inputs.shape: torch.Size([128, 14, 49])
labels.shape: torch.Size([128, 28])
Make Nongransformer ==============================
Complete Param Init ==============================
0 Epoch ==================================================
No Update Count: 1 / 1


100%|██████████| 329/329 [00:26<00:00, 12.22it/s]


Train Total Loss: 1669.59088077066
gradient flow good


100%|██████████| 83/83 [00:02<00:00, 35.47it/s]


Eval Total Loss: 1909.5960526540791
Best Loss: 20066035.321289062
1 Epoch ==================================================
Update Count: 1 / 2


...

Train Total Loss: 1628.7582969733655
gradient flow good


100%|██████████| 83/83 [00:02<00:00, 37.29it/s]


Eval Total Loss: 1908.9638943245623
No Update
73 Epoch ==================================================
No Update Count: 66 / 74


 64%|██████▎   | 209/329 [00:17<00:09, 12.26it/s]


KeyboardInterrupt: ignored